# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use graphlab SArray and SFrame functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

# Fire up graphlab create

In [1]:
import graphlab

# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1449464053 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to C:\Python27\lib\site-packages\certifi\cacert.pem
1449464053 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to vthukral@gmail.com and will expire on September 26, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-12108 - Server binary: C:\Python27\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\vaibhavt\AppData\Local\Temp\graphlab_server_1449464053.log.0
[INFO] GraphLab Server Version: 1.7.1


# Split data into training and testing

We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

# Useful SFrame summary functions

In order to make use of the closed form soltion as well as take advantage of graphlab's built in functions we will review some important ones. In particular:
* Computing the sum of an SArray
* Computing the arithmetic average (mean) of an SArray
* multiplying SArrays by constants
* multiplying SArrays by other SArrays

In [4]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = sales['price'] # extract the price column of the sales SFrame -- this is now an SArray

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
sum_prices = prices.sum()
num_houses = prices.size() # when prices is an SArray .size() returns its length
avg_price_1 = sum_prices/num_houses
avg_price_2 = prices.mean() # if you just want the average, the .mean() function
print "average price via method 1: " + str(avg_price_1)
print "average price via method 2: " + str(avg_price_2)

average price via method 1: 540088.141905
average price via method 2: 540088.141905


As we see we get the same answer both ways

In [5]:
# if we want to multiply every price by 0.5 it's a simple as:
half_prices = 0.5*prices
# Let's compute the sum of squares of price. We can multiply two SArrays of the same length elementwise also with *
prices_squared = prices*prices
sum_prices_squared = prices_squared.sum() # price_squared is an SArray of the squares and we want to add them up.
print "the sum of price squared is: " + str(sum_prices_squared)

the sum of price squared is: 9.21732513355e+15


Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

# Build a generic simple linear regression function 

Armed with these SArray functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as SArrays: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [8]:
def simple_linear_regression(input_feature, output):
    # compute the mean of  input_feature and output
    scalar_mean_xi = input_feature.mean()
    scalar_mean_yi = output.mean()
    
    # compute the product of the output and the input_feature and its mean
    vec_prod_xi_yi = input_feature*output
    
    # compute the squared value of the input_feature and its mean
    vec_sqr_xi_xi = input_feature*input_feature
    
    # use the formula for the slope
    numerator = vec_prod_xi_yi.sum() - input_feature.sum() * scalar_mean_yi;
    denominator = vec_sqr_xi_xi.sum() - (input_feature.sum()*input_feature.sum()/input_feature.size())
    slope = numerator/denominator
    
    # use the formula for the intercept
    intercept = scalar_mean_yi - slope*scalar_mean_xi;
    
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [9]:
test_feature = graphlab.SArray(range(5))
test_output = graphlab.SArray(1 + 1*test_feature)
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print "Intercept: " + str(test_intercept)
print "Slope: " + str(test_slope)

Intercept: 1.0
Slope: 1.0


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [10]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

print "Intercept: " + str(sqft_intercept)
print "Slope: " + str(sqft_slope)

Intercept: -47116.0765749
Slope: 281.958838568


# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using SArrays it's easy to multiply an SArray by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [11]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    predicted_values = intercept + input_feature*slope
    
    return predicted_values

Now that we can calculate a prediction given the slop and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [16]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print "The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price)

The estimated price for a house with 2650 squarefeet is $700074.85


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [31]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    #print input_feature
    #print output
    #print intercept
    #print slope
    
    # First get the predictions
    output_hat = input_feature * slope + intercept

    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)
    residuals = output - output_hat
    print 'residuals = ', residuals

    # square the residuals and add them up
    residuals_sqr = residuals*residuals
    print 'residual_sqr = ', residuals
    
    RSS = residuals_sqr.sum()
    print 'RSS = ', RSS

    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [32]:
print get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope) # should be 0.0

residuals =  [0.0, 0.0, 0.0, 0.0, 0.0]
residual_sqr =  [0.0, 0.0, 0.0, 0.0, 0.0]
RSS =  0.0
0.0


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [33]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft)

residuals =  [-63695.35293494281, -139518.1385440426, 10007.770877813251, 98476.75298225298, 83425.22778120835, -256100.8284619809, -178943.3315686611, 40089.70769318094, -225270.65607556142, -162786.12831800815, -294157.3887260633, 188043.82383641117, 60832.467737194616, 66770.57876740769, -90786.12831800815, 80981.93486662419, -102234.52970629674, -75332.47163468163, -24657.241904729803, 1187141.6189434626, -307930.4869737333, -1879.8806924960809, -314683.0779159189, 293316.9220840811, 319373.7025801637, 56538.641952039965, 41481.861455957405, -8414.941320619779, 77651.98271220492, -211219.1308745168, 94610.74786923581, 129064.40455256234, 21693.217555173906, -56873.70647765073, -52518.138544042595, -104535.94807593268, 289440.7000236551, -4791.236791712581, -355873.85329898424, -21533.173607080244, 186430.33625491278, 144667.52836531837, 620883.3939980688, -60414.94132061978, -496054.26491330727, 249554.11419448676, 73837.57621089905, 197368.44728512585, 134172.56342835608, -334224.

# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Comlplete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output!

In [34]:
def inverse_regression_predictions(output, intercept, slope):
    # solve output = intercept + slope*input_feature for input_feature. Use this equation to compute the inverse predictions:
    estimated_feature = (output - intercept)/slope
    
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that coses $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [35]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print "The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet)

The estimated squarefeet for a house worth $800000.00 is 3004


# New Model: estimate prices from bedrooms

We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [36]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'

bdr_intercept, bdr_slope = simple_linear_regression(train_data['bedrooms'], train_data['price'])

print "Intercept: " + str(bdr_intercept)
print "Slope: " + str(bdr_slope)


Intercept: 109473.180469
Slope: 127588.952175


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

In [37]:
# Compute RSS when using bedrooms on TEST data:
rss_prices_on_bdr = get_residual_sum_of_squares(test_data['bedrooms'], test_data['price'], bdr_intercept, bdr_slope)
print 'The RSS of predicting Prices based on bedrooms is : ' + str(rss_prices_on_bdr)


residuals =  [-182240.03699303942, 30171.010832376312, -259240.03699303942, 88259.96300696058, 42759.96300696058, -14828.989167623688, 155171.0108323763, -327328.9891676237, -203240.03699303942, -48828.98916762369, -143240.03699303942, -259828.9891676237, -376328.9891676237, -244740.03699303942, -277417.94134220807, -139828.9891676237, 277759.9630069606, 27709.963006960577, -219717.94134220807, -72240.03699303942, 270171.0108323763, -209290.03699303942, -492417.94134220807, -72240.03699303942, 187271.0108323763, -208240.03699303942, 297671.0108323763, -67240.03699303942, -232240.03699303942, -195240.03699303942, -217990.03699303942, 220171.0108323763, 185659.96300696058, -84651.08481845516, -212290.03699303942, 115298.91518154484, -354878.9891676237, 277582.05865779193, 1780171.0108323763, -232240.03699303942, 32759.963006960577, 132837.86735612917, -282240.0369930394, -314928.9891676237, -194651.08481845516, 1302582.058657792, 160171.0108323763, -162240.03699303942, -259828.9891676237

In [38]:
# Compute RSS when using squarfeet on TEST data:
rss_prices_on_sqft = get_residual_sum_of_squares(test_data['sqft_living'], test_data['price'], sqft_intercept, sqft_slope)
print 'The RSS of predicting Prices based on sqft is : ' + str(rss_prices_on_sqft)


residuals =  [-46085.06257686723, -134662.49719976773, -202033.53737582255, -26528.428902118118, 274780.94253615, -86616.08047242742, -367750.2221807437, -294791.31020237936, -19152.060020358593, -157270.7294862282, -108590.24446123838, -128605.71670368512, -267662.42378910095, -258023.24701774702, -55260.36571748578, -383610.971998723, -3384.143657059758, 237174.23545073415, -220307.84818596696, 168239.70769318094, 218121.03822731134, -22382.471634681628, -278719.1308745168, -102440.77733180881, 98566.38921351067, -108739.7115906679, -129384.21706772665, 156322.17737911904, -163755.1838331147, -202884.0702463931, 64783.533478335565, 187858.1569270502, 37036.51046975807, 87451.06379239744, -166361.89091853047, 101308.23033771699, -172903.12576149957, 11950.84356039716, 1417966.3158028438, -203229.4212325923, -19997.484417224652, 202727.35926349022, -222213.94899014558, -381076.9037010735, -25368.52459327955, 1017213.7248606584, -159739.8584120014, -305224.3127588879, -201915.0147312865